<a href="https://colab.research.google.com/github/AdamPeetz/PlaneGAN/blob/main/MSDS696_GAN_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# example of a dcgan on cifar10
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import tensorflow as tf
import cv2
import numpy as np
from tqdm import tqdm
import pathlib

import os, shutil 
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 16x16
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 8x8
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 4x4
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model


In [4]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [5]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [6]:
# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	(trainX, _), (_, _) = load_data()
	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

#load and prepare custom image dataset
def create_real_img_dataset(image_path, resize_dim):
		dataset = []
		for file in tqdm(os.listdir(image_path)):
				image = cv2.imread(os.path.join(image_path,file))
				X = image.astype('float32')
				X = X[:, :, [2, 1, 0]]
				X = (X - 127.5) / 127.5
				X = tf.image.resize(X, [resize_dim, resize_dim])
				dataset.append(X)
		dataset = np.array(dataset)
		return dataset

#load a custom image dataset with 1 level of directory
# tensorflow documentation
def load_custom_image_set_one_level(dataroot,resizedim):
    #image parse function
    def parse_image(filename):
        parts = tf.strings.split(filename, os.sep)
        label = parts[-2]
        image = tf.io.read_file(filename)
        image = tf.io.decode_jpeg(image)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, [resizedim,resizedim])
        return image, label
    
    data_root = pathlib.Path(dataroot)
    list_ds = tf.data.Dataset.list_files(str(data_root/'*/*'))
    images_ds = list_ds.map(parse_image)
    #https://stackoverflow.com/questions/70535683/extract-data-from-tensorflow-dataset-e-g-to-numpy
    images = np.asarray(list(images_ds.map(lambda x, y: x)))
    return images

# load a custom image dataset with 2 levels of directory
# tensorflow documentation
def load_custom_image_set_two_levels(dataroot,resizedim):
    #image parse function
    def parse_image(filename):
        parts = tf.strings.split(filename, os.sep)
        label = parts[-2]
        image = tf.io.read_file(filename)
        image = tf.io.decode_jpeg(image)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, [resizedim,resizedim])
        image = (image - 127.5) / 127.5
        return image, label
    
    data_root = pathlib.Path(dataroot)
    list_ds = tf.data.Dataset.list_files(str(data_root/'*/*/*'))
    images_ds = list_ds.map(parse_image)
    #https://stackoverflow.com/questions/70535683/extract-data-from-tensorflow-dataset-e-g-to-numpy
    images = np.asarray(list(images_ds.map(lambda x, y: x)))
    return images

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# modified with suggestion from Das Shuvo (2020)
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	input_tensor = tf.convert_to_tensor(x_input)
	X = g_model(input_tensor)
	X = X.numpy()
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [16]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
#	examples = (examples + 1) / 2.0
	example = (examples + 1) * 255
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = '/content/gdrive/My Drive/planegan/sample_output/generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()
 
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d1_hist, label='d-real')
	pyplot.plot(d2_hist, label='d-fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# plot discriminator accuracy
	pyplot.subplot(2, 1, 2)
	pyplot.plot(a1_hist, label='acc-real')
	pyplot.plot(a2_hist, label='acc-fake')
	pyplot.legend()
	# save plot to file
	pyplot.savefig('/content/gdrive/My Drive/planegan/saved_models/plot_line_plot_loss.png')
	pyplot.close()


In [17]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = '/content/gdrive/My Drive/planegan/saved_models/generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=300, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# define half batches for training
	half_batch = int(n_batch / 2)
  # prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
	 	  # record training metrics
			d1_hist.append(d_loss1)
			d2_hist.append(d_loss2)
			g_hist.append(g_loss)
			a1_hist.append(d_acc1)
			a2_hist.append(d_acc2)
		# summarize loss
			if (j+1) % 5 == 0:
				print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)
	# save plot to directory
	plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)


In [18]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
image_path = "/content/gdrive/My Drive/cars_tanks/"
resizedim = 32
dataset = load_custom_image_set_two_levels(image_path,resizedim)
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 5/11, d1=0.301, d2=0.807 g=0.596
>1, 10/11, d1=0.379, d2=0.949 g=0.657
>2, 5/11, d1=0.716, d2=0.514 g=1.031
>2, 10/11, d1=0.300, d2=0.738 g=0.689
>3, 5/11, d1=0.338, d2=1.107 g=0.527
>3, 10/11, d1=0.557, d2=0.698 g=0.822
>4, 5/11, d1=0.330, d2=1.080 g=0.562
>4, 10/11, d1=0.572, d2=0.663 g=0.916
>5, 5/11, d1=0.516, d2=0.957 g=0.635
>5, 10/11, d1=0.442, d2=0.543 g=1.047
>6, 5/11, d1=0.445, d2=1.101 g=0.497
>6, 10/11, d1=0.541, d2=0.799 g=0.806
>7, 5/11, d1=0.602, d2=0.847 g=0.706
>7, 10/11, d1=0.605, d2=0.861 g=0.636
>8, 5/11, d1=0.602, d2=0.833 g=0.681
>8, 10/11, d1=0.678, d2=0.701 g=0.769
>9, 5/11, d1=0.633, d2=0.806 g=0.678
>9, 10/11, d1=0.653, d2=0.633 g=0.887
>10, 5/11, d1=0.528, d2=0.593 g=1.038
>10, 10/11, d1=0.723, d2=0.969 g=0.645


>Accuracy real: 35%, fake: 0%


>11, 5/11, d1=0.704, d2=0.520 g=1.022
>11, 10/11, d1=0.602, d2=0.633 g=0.884
>12, 5/11, d1=0.678, d2=0.865 g=0.678
>12, 10/11, d1=0.671, d2=0.498 g=1.097
>13, 5/11, d1=0.624, d2=0.701 g=0.800
>13, 10/11, d1=0.640, d2=0.840 g=0.685
>14, 5/11, d1=0.683, d2=0.616 g=0.876
>14, 10/11, d1=0.686, d2=0.582 g=1.009
>15, 5/11, d1=0.689, d2=0.678 g=0.892
>15, 10/11, d1=0.622, d2=0.729 g=0.748
>16, 5/11, d1=0.568, d2=0.692 g=0.790
>16, 10/11, d1=0.597, d2=0.733 g=0.874
>17, 5/11, d1=0.642, d2=0.643 g=0.967
>17, 10/11, d1=0.756, d2=0.645 g=0.937
>18, 5/11, d1=0.654, d2=0.633 g=0.885
>18, 10/11, d1=0.580, d2=0.675 g=0.988
>19, 5/11, d1=0.816, d2=0.628 g=0.952
>19, 10/11, d1=0.811, d2=0.618 g=0.862
>20, 5/11, d1=0.745, d2=0.655 g=0.810
>20, 10/11, d1=0.728, d2=0.689 g=0.778


>Accuracy real: 23%, fake: 96%


>21, 5/11, d1=0.706, d2=0.636 g=0.804
>21, 10/11, d1=0.647, d2=0.696 g=0.736
>22, 5/11, d1=0.583, d2=0.815 g=0.674
>22, 10/11, d1=0.708, d2=0.669 g=0.794
>23, 5/11, d1=0.742, d2=0.500 g=1.039
>23, 10/11, d1=0.719, d2=0.582 g=0.885
>24, 5/11, d1=0.652, d2=0.711 g=0.767
>24, 10/11, d1=0.659, d2=0.667 g=0.899
>25, 5/11, d1=0.693, d2=0.742 g=0.743
>25, 10/11, d1=0.609, d2=0.726 g=0.719
>26, 5/11, d1=0.544, d2=0.652 g=0.860
>26, 10/11, d1=0.613, d2=0.612 g=1.020
>27, 5/11, d1=0.755, d2=0.621 g=1.011
>27, 10/11, d1=0.644, d2=0.556 g=1.051
>28, 5/11, d1=0.606, d2=0.825 g=0.711
>28, 10/11, d1=0.636, d2=0.646 g=0.855
>29, 5/11, d1=0.732, d2=0.679 g=0.831
>29, 10/11, d1=0.657, d2=0.694 g=0.761
>30, 5/11, d1=0.481, d2=0.904 g=0.637
>30, 10/11, d1=0.564, d2=0.748 g=0.886


...

>Accuracy real: 58%, fake: 83%


>281, 5/11, d1=0.551, d2=0.479 g=1.250
>281, 10/11, d1=0.722, d2=0.768 g=1.330
>282, 5/11, d1=0.591, d2=0.469 g=1.277
>282, 10/11, d1=0.487, d2=0.622 g=1.230
>283, 5/11, d1=0.614, d2=0.647 g=1.255
>283, 10/11, d1=0.803, d2=0.637 g=1.148
>284, 5/11, d1=0.771, d2=0.553 g=1.166
>284, 10/11, d1=0.631, d2=0.550 g=1.182
>285, 5/11, d1=0.546, d2=0.667 g=1.166
>285, 10/11, d1=0.592, d2=0.559 g=1.219
>286, 5/11, d1=0.641, d2=0.531 g=1.153
>286, 10/11, d1=0.315, d2=0.509 g=1.139
>287, 5/11, d1=0.369, d2=0.561 g=1.158
>287, 10/11, d1=0.597, d2=0.401 g=1.465
>288, 5/11, d1=0.648, d2=0.496 g=1.452
>288, 10/11, d1=0.647, d2=0.517 g=1.294
>289, 5/11, d1=0.587, d2=0.537 g=1.306
>289, 10/11, d1=0.518, d2=0.471 g=1.208
>290, 5/11, d1=0.520, d2=0.490 g=1.231
>290, 10/11, d1=0.520, d2=0.571 g=1.190


>Accuracy real: 72%, fake: 71%


>291, 5/11, d1=0.982, d2=0.568 g=1.357
>291, 10/11, d1=0.687, d2=0.599 g=1.244
>292, 5/11, d1=0.581, d2=0.532 g=1.086
>292, 10/11, d1=0.431, d2=0.529 g=1.253
>293, 5/11, d1=0.332, d2=0.517 g=1.401
>293, 10/11, d1=0.581, d2=0.450 g=1.263
>294, 5/11, d1=0.349, d2=0.481 g=1.309
>294, 10/11, d1=0.564, d2=0.664 g=1.211
>295, 5/11, d1=0.599, d2=0.427 g=1.682
>295, 10/11, d1=0.627, d2=0.510 g=1.349
>296, 5/11, d1=0.637, d2=0.446 g=1.456
>296, 10/11, d1=0.591, d2=0.548 g=1.394
>297, 5/11, d1=0.712, d2=0.512 g=1.310
>297, 10/11, d1=0.584, d2=0.386 g=1.565
>298, 5/11, d1=0.451, d2=0.426 g=1.733
>298, 10/11, d1=0.695, d2=0.703 g=1.493
>299, 5/11, d1=0.928, d2=0.291 g=1.698
>299, 10/11, d1=0.459, d2=0.467 g=1.455
>300, 5/11, d1=0.451, d2=0.555 g=1.463
>300, 10/11, d1=0.698, d2=0.488 g=1.249


>Accuracy real: 57%, fake: 86%


In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# create and save a plot of generated images
def save_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :])
	pyplot.show()

# load model
model = gan_model
# generate images
latent_points = generate_latent_points(100, 100)
# generate images
X = model.predict(latent_points)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, 10)

# References

Brownlee, Jason. (2019). Generative Adversarial Networks with Python. Ebook. Machine Learning Mastery. 

Das Shuvo, Falguni. (2020). Repeatedly calling model.predict(...) results in memory leak. GitHub. Retrieved 3/15/2023 from https://github.com/keras-team/keras/issues/13118